In [ ]:
from datetime import timedelta

from adcircpy import AdcircMesh, AdcircRun, Tides
from adcircpy.forcing.winds import BestTrackForcing
from adcircpy.server import SlurmConfig

# load an ADCIRC mesh grid from a `fort.14` file to a new mesh object
mesh = AdcircMesh.open("fort.14", crs="epsg:4326")

# add nodal attributes from a `fort.13` file to the mesh object
mesh.import_nodal_attributes("fort.13")

# create a tidal forcing object, using all constituents
tidal_forcing = Tides()
# tidal_forcing.use_all() # get rid of tides.

# add data from the tidal forcing object to the mesh object
mesh.add_forcing(tidal_forcing)

# create a wind forcing object for Hurricane Sandy (2012)
wind_forcing = BestTrackForcing("Sandy2012")


# add wind forcing data to the mesh object
mesh.add_forcing(wind_forcing)

# create a Slurm (HPC job manager) configuration object.
slurm = SlurmConfig(
    account="n01-SOWISE",
    ntasks=128,
    run_name="ADCIRCpy-sandy2012",
    partition="standard",
    walltime=timedelta(minutes=20),
    mail_type="all",
    mail_user="sdat2@cam.ac.uk",
    log_filename="example.log",
    modules=[
        "PrgEnv-gnu/8.3.3",
        "cray-hdf5-parallel/1.12.2.1",
        "cray-netcdf-hdf5parallel/4.9.0.1",
    ],
    path_prefix="/work/n01/n01/sithom/adcirc-swan/0/adcirc/work",
)

# create an ADCIRC run driver object
driver = AdcircRun(
    mesh=mesh,
    server_config=slurm,
    spinup_time=timedelta(days=15),
    #output_interval=timedelta(hours=1),
    #output_variables=["zeta", "u", "v"],
    output_netcdf=True,
    output_directory="./model_outputs_sandy2012",
    # output_prefix="sandy2012",
)

# write configuration files to the specified directory
driver.write(output_directory="./model_inputs_sandy2012")

: 

In [ ]:
AdcircRun

: 